# Projet : Détection de la dépression chez les étudiants

**Objectif** : Prédire si une personne est en dépression à partir de données cliniques et socio-professionnelles.

---

## Métriques de performance

Les classes sont **fortement déséquilibrées** : 18% de cas de dépression, 82% de cas non-dépressifs.

- **Recall (sensibilité)** : TP / (TP + FN) → éviter de **manquer un cas positif**  
- **Precision (précision)** : TP / (TP + FP) → limiter les **faux positifs**  
- **F1-score** → compromis entre précision et recall

**Objectif ciblé :** F1 ≈ 0.5 et Recall ≈ 0.7

---

## Résumé des principaux insights (EDA)

- Classes déséquilibrées : 18% dépressifs / 82% non-dépressifs  
- Étudiants plus touchés que les travailleurs  
- Tranche 20-30 ans particulièrement affectée  
- Pression académique et professionnelle plus élevée chez les déprimés  
- Hygiène de vie et Financial Stress semblent influencer la dépression  
- Lien fort entre idées suicidaires et dépression  
- Certaines variables ont >80% de NaN → à traiter avant modélisation

## Imports et Configuration

In [ ]:
import sys
import os

# Ajouter le dossier parent au path pour importer src
sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Modules personnalisés
from src.eda import (
    # Analyse de la forme
    get_data_shape, get_dtypes_summary, plot_missing_values, get_missing_values_summary,
    # Analyse du fond
    plot_target_distribution, plot_continuous_distributions, plot_categorical_distributions,
    print_unique_values, create_target_subsets,
    # Relations features-target
    plot_feature_by_target, plot_multiple_features_by_target,
    plot_categorical_by_target, plot_crosstab_heatmap, plot_multiple_crosstabs,
    # Analyse complémentaire
    plot_correlation_matrix, get_correlation_with_column, plot_feature_by_category,
    # Transformations
    categorize_sleep_duration, categorize_dietary_habits, categorize_sleep_numeric,
    # Analyse NaN
    analyze_nan_impact, count_remaining_after_dropna,
    # Tests statistiques
    perform_ttest
)

import warnings
warnings.filterwarnings("ignore")

# Configuration
TRAIN_PATH = '../data/train.csv'
TARGET_FEATURE = 'Depression'

print("Modules chargés avec succès !")

## Chargement des données

In [ ]:
data = pd.read_csv(TRAIN_PATH)
df = data.copy()
df.head()

---
# 1. Analyse de la forme

## 1.1 Identification de la target et dimensions

In [ ]:
print(f"Variable cible: {TARGET_FEATURE}")
print()
data_info = get_data_shape(df)

## 1.2 Types de variables

In [ ]:
get_dtypes_summary(df)

## 1.3 Valeurs manquantes

In [ ]:
plot_missing_values(df)

In [ ]:
missing_summary = get_missing_values_summary(df)

### 1.4 Informations complémentaires

- Nombre de lignes et colonnes : 140,700 × 20  
- Groupes de données :
  - ~80% → études (`Academic Pressure`, `CGPA`, `Study Satisfaction`)  
  - ~20% → emploi (`Profession`, `Work Pressure`, `Job Satisfaction`)

---
# 2. Analyse du Fond

### Elimination des colonnes inutiles

In [ ]:
df.drop('id', axis=1, inplace=True)
print(f"Colonne 'id' supprimée. Nouvelles dimensions: {df.shape}")

## 2.1 Visualisation de la target

In [ ]:
target_dist = plot_target_distribution(df, TARGET_FEATURE)

## 2.2 Compréhension des variables

### A. Variables continues

In [ ]:
float_columns = df.select_dtypes('float').columns.tolist()
print(f"Variables continues: {float_columns}")

plot_continuous_distributions(df, float_columns)

### B. Variables qualitatives

In [ ]:
object_columns = df.select_dtypes('object').columns.tolist()
print(f"Variables qualitatives: {object_columns}")

In [ ]:
print_unique_values(df, object_columns, max_display=20)

In [ ]:
plot_categorical_distributions(df, object_columns, top_n=10)

## 2.3 Relations features-target

### Création de sous-ensembles positif et négatif

In [ ]:
positive_df, negative_df = create_target_subsets(df, TARGET_FEATURE)

### Définition des groupes de colonnes

In [ ]:
job_columns = ['Work Pressure', 'Job Satisfaction']
studies_columns = ['Academic Pressure', 'CGPA', 'Study Satisfaction']
binary_columns = ['Have you ever had suicidal thoughts ?', 'Family History of Mental Illness', 
                  'Gender', 'Working Professional or Student']

print(f"Colonnes job: {job_columns}")
print(f"Colonnes études: {studies_columns}")
print(f"Colonnes binaires: {binary_columns}")

### A. Job / target

In [ ]:
plot_categorical_by_target(df, 'Profession', TARGET_FEATURE, top_n=10)

In [ ]:
plot_multiple_features_by_target(df, job_columns, TARGET_FEATURE)

**Observations:**
- Work Pressure plus élevée chez les déprimés
- Job Satisfaction plus faible chez les déprimés

### B. Age / target

In [ ]:
plot_feature_by_target(df, 'Age', TARGET_FEATURE)

**Observation:** Les 20-30 ans semblent plus affectés par la dépression

### C. Studies / target

In [ ]:
plot_multiple_features_by_target(df, studies_columns, TARGET_FEATURE)

**Observations:**
- Academic Pressure plus forte chez les étudiants déprimés
- Study Satisfaction plus faible chez les déprimés

### D. Hygiène de vie / target

In [ ]:
lifestyle_columns = ['Sleep Duration', 'Dietary Habits', 'Degree']

for col in lifestyle_columns:
    plot_categorical_by_target(df, col, TARGET_FEATURE, top_n=10)

### E. Variables binaires / target

In [ ]:
plot_multiple_crosstabs(df, binary_columns, TARGET_FEATURE)

**Observation majeure:** Fort lien entre idées suicidaires et dépression

---
# 3. Analyse du fond complémentaire

## 3.1 Relation variables / variables

### A. Corrélation Float / Float

In [ ]:
corr_matrix = plot_correlation_matrix(df)

### B. Corrélation Age / autres variables

In [ ]:
age_correlations = get_correlation_with_column(df, 'Age')

**Observation:** Très faible corrélation entre l'âge et les autres variables

### C. Sommeil / Float

In [ ]:
# Catégorisation du sommeil
df = categorize_sleep_duration(df)
print("Catégories de sommeil créées:")
print(df['sommeil'].value_counts())

In [ ]:
for col in float_columns:
    plot_feature_by_category(df, col, 'sommeil')

### D. Working Professional or Student / Float

In [ ]:
for col in float_columns:
    plot_feature_by_category(df, col, 'Working Professional or Student')

**Observations:**
- 20-35 ans : majoritairement étudiants
- 35-60 ans : majoritairement travailleurs

## 3.2 Analyse des valeurs manquantes (NaN)

In [ ]:
count_remaining_after_dropna(df)

In [ ]:
# Analyse impact NaN pour les colonnes job
df_job = df[job_columns + [TARGET_FEATURE]].dropna()
print(f"Avec job_columns uniquement:")
print(f"  {len(df_job)} lignes")
print(f"  Distribution de {TARGET_FEATURE}:")
print(df_job[TARGET_FEATURE].value_counts(normalize=True))

In [ ]:
# Analyse impact NaN pour les colonnes études
df_studies = df[studies_columns + [TARGET_FEATURE]].dropna()
print(f"Avec studies_columns uniquement:")
print(f"  {len(df_studies)} lignes")
print(f"  Distribution de {TARGET_FEATURE}:")
print(df_studies[TARGET_FEATURE].value_counts(normalize=True))

**Observation importante:** Les étudiants (avec Academic Pressure, CGPA, Study Satisfaction) ont un taux de dépression beaucoup plus élevé (~58%) que les professionnels (~8%)

## 3.3 Hypothèses nulles (Tests statistiques)

### Préparation des variables pour les tests

In [ ]:
# Conversion des variables catégorielles en numériques
df = categorize_dietary_habits(df)
df = categorize_sleep_numeric(df)

print("Variables créées pour les tests:")
print(f"  - diet_float: {df['diet_float'].value_counts().to_dict()}")
print(f"  - sommeil_float: {df['sommeil_float'].value_counts().to_dict()}")

### Test t de Student

**H0:** La durée de sommeil / régime alimentaire est ÉGALE chez les individus déprimés et non déprimés

In [ ]:
# Test pour le sommeil
print("=" * 50)
sleep_test = perform_ttest(df, 'sommeil_float', TARGET_FEATURE)

In [ ]:
# Test pour le régime alimentaire
print("=" * 50)
diet_test = perform_ttest(df, 'diet_float', TARGET_FEATURE)

In [ ]:
# Tests pour les autres variables
print("=" * 50)
for col in ['Age', 'Financial Stress', 'Work/Study Hours']:
    perform_ttest(df, col, TARGET_FEATURE)
    print()

---
# Bilan de l'EDA

## Résumé des insights clés

### Forme des données
- **140,700 observations** × **20 variables**
- 10 variables qualitatives, 10 quantitatives
- 3 variables avec >80% de NaN (colonnes études)

### Distribution de la target
- **18% dépressifs** / **82% non-dépressifs** → Classes déséquilibrées

### Facteurs de risque identifiés
1. **Statut** : Étudiants plus touchés que les travailleurs
2. **Âge** : 20-30 ans particulièrement affectés
3. **Pression** : Academic/Work Pressure élevée corrélée à la dépression
4. **Satisfaction** : Study/Job Satisfaction faible chez les déprimés
5. **Hygiène de vie** : Sommeil et alimentation influencent la dépression
6. **Stress financier** : Plus élevé chez les déprimés
7. **Idées suicidaires** : Fort lien avec la dépression

### Points d'attention pour la modélisation
- Traitement des valeurs manquantes (>80% pour certaines colonnes)
- Gestion du déséquilibre des classes
- Combinaison des colonnes étudiants/professionnels (Pressure, Satisfaction)